<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 02 - Data Virtualization

In this tutorial you will learn how to create and query Virtualized Data in a SQL Server big data cluster. 

**TODO:** Complete Tutorial 


In [2]:
/* Create External File Format */

USE WideWorldImporters;
GO

IF NOT EXISTS(SELECT * FROM sys.external_file_formats WHERE name = 'csv_file')
BEGIN
    CREATE EXTERNAL FILE FORMAT csv_file
    WITH (
        FORMAT_TYPE = DELIMITEDTEXT,
        FORMAT_OPTIONS(
            FIELD_TERMINATOR = ',',
            STRING_DELIMITER = '0x22',
            FIRST_ROW = 2,
            USE_TYPE_DEFAULT = TRUE)
    );
END

Commands completed successfully.

Total execution time: 00:00:00.0076252

Commands completed successfully.

Total execution time: 00:00:00.0203934

In [2]:
/* Create External Data Source to the Storage Pool */
IF NOT EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlStoragePool')
BEGIN
    CREATE EXTERNAL DATA SOURCE SqlStoragePool
    WITH (LOCATION = 'sqlhdfs://service-master-pool:50070');
END

Commands completed successfully.

Total execution time: 00:00:00.4077123

In [3]:
/* Create an External Table that can read from the Storage Pool File Location */
IF NOT EXISTS(SELECT * FROM sys.external_tables WHERE name = 'web_customers_hdfs')
BEGIN
    CREATE EXTERNAL TABLE [web_customers_hdfs]
    ("CustomerCategoryName" VARCHAR(250) 
    , "PrimaryContact" VARCHAR(250) 
    , "WebsiteURL" VARCHAR(250))
    WITH
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/adventureworks_export',
        FILE_FORMAT = csv_file
    );
END

Commands completed successfully.

Total execution time: 00:00:01.7761457

In [3]:
/* Read Data from HDFS using only T-SQL */

SELECT TOP 10 hdfs.PrimaryContact
, hdfs.CustomerCategoryName
    FROM [web_customers_hdfs] hdfs
WHERE WebsiteURL LIKE '%wingtip%';
GO


In [4]:
/* Now Join Those to show customers we currently have in a SQL Server Database 
and the Category they qre in the External Table */

SELECT TOP 10 a.FullName, b.CustomerCategoryName
  FROM [WideWorldImporters].[Application].[People] a
  INNER JOIN [WideWorldImporters].[dbo].[web_customers_hdfs] b  ON a.FullName = b.PrimaryContact;
  GO

## Next Steps: Continue on to Working with the SQL Server Data Pool